## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-19-17-49-00 +0000,wsj,Israel Hits Dozens of Targets in Gaza After Sa...,https://www.wsj.com/world/middle-east/hamas-at...
1,2025-10-19-17-45-26 +0000,startribune,Live: Vikings trail 14-3 at home to the Super ...,https://www.startribune.com/minnesota-vikings-...
2,2025-10-19-17-45-00 +0000,wsj,Heist at Louvre Leaves Museum Missing Priceles...,https://www.wsj.com/world/europe/seven-minute-...
3,2025-10-19-17-38-31 +0000,nypost,New Jersey GOP sounds alarms about ballot secu...,https://nypost.com/2025/10/19/us-news/nj-gop-s...
4,2025-10-19-17-32-47 +0000,nyt,Vance Flexes Marines’ Might as Mass Protests A...,https://www.nytimes.com/2025/10/19/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,31
350,no,17
351,kings,16
4,gaza,12
0,israel,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
55,2025-10-19-12-09-54 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...,93
97,2025-10-19-06-11-20 +0000,bbc,Millions turned out for anti-Trump 'No Kings' ...,https://www.bbc.com/news/articles/c93xgyp1zv4o...,91
160,2025-10-18-18-45-48 +0000,cbc,'Trump must go': Large crowds join 'No Kings' ...,https://www.cbc.ca/news/world/no-kings-protest...,90
49,2025-10-19-12-33-49 +0000,nypost,Trump posts wild AI video showing him flying f...,https://nypost.com/2025/10/19/us-news/trump-po...,89
109,2025-10-19-03-33-45 +0000,missionlocal,"Photos: Defiance, solidarity fill S.F. streets...",https://missionlocal.org/2025/10/photos-defian...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
55,93,2025-10-19-12-09-54 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...
48,56,2025-10-19-12-41-57 +0000,nypost,Israel launches ‘extensive’ Gaza airstrikes in...,https://nypost.com/2025/10/19/world-news/israe...
15,39,2025-10-19-17-06-24 +0000,nypost,Video shows Louvre thief cutting into jewel ca...,https://nypost.com/2025/10/19/world-news/louvr...
27,36,2025-10-19-16-10-25 +0000,bbc,Trump ends aid to Colombia and calls country's...,https://www.bbc.com/news/articles/cn8xg1jve73o...
96,35,2025-10-19-06-31-02 +0000,bbc,Afghanistan pulls out of cricket series after ...,https://www.bbc.com/news/articles/c20pnz01x0eo...
18,27,2025-10-19-16-59-32 +0000,nypost,"Curtis Sliwa bashes Andrew Cuomo, Dems for pus...",https://nypost.com/2025/10/19/us-news/sliwa-ri...
57,27,2025-10-19-12-00-00 +0000,wsj,The GOP is reviving a culture war strategy use...,https://www.wsj.com/politics/elections/republi...
20,25,2025-10-19-16-48-19 +0000,nypost,Lying George Santos already back on Cameo seek...,https://nypost.com/2025/10/19/us-news/lying-ge...
118,23,2025-10-19-01-19-22 +0000,nypost,Twisted NY man who bragged he was planning to ...,https://nypost.com/2025/10/18/us-news/david-co...
73,23,2025-10-19-09-57-15 +0000,nypost,"Scooter-riding thieves break into Louvre, swip...",https://nypost.com/2025/10/19/world-news/louvr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
